## 1. Configurações e Importações

In [47]:
import pandas as pd
import numpy as np 
import re 
import os 
import unicodedata
import psycopg2
from psycopg2.extras import execute_batch


In [48]:
INPUT_FILE = '../Data Layer/raw/fifa21_raw_data.csv'
OUTPUT_DIR = '../Data Layer/silver'
OUTPUT_FILE_CSV = 'fifa21_silver.csv'

DB_CONFIG = {
    'host': 'localhost',
    'port': 5433,
    'database': 'airflow',
    'user': 'airflow',
    'password': 'airflow'
}

## 2. Funções de transformações

### 2.1 Conversão de M e K

Converte k(milhar) em 1.000 e m(milhão) em 1.000.000

In [49]:
def convert_m_and_k(value):
    if pd.isna(value) or value == '': return 0.0
    val = str(value).replace('€', '').strip()
    multiplier = 1
    if 'M' in val:
        multiplier = 1000000
        val = val.replace('M', '')
    elif 'K' in val:
        multiplier = 1000
        val = val.replace('K', '')
    try:
        return float(val) * multiplier
    except:
        return 0.0

### 2.2 Converte altura de Foot para Centímetros 

In [50]:
def convert_height_cm(h):
    if pd.isna(h):
        return None
    try:
        parts = re.findall(r"\d+", str(h))
        feet = int(parts[0])
        inches = int(parts[1]) if len(parts) > 1 else 0
        return round((feet * 30.48) + (inches * 2.54), 1)
    except:
        return None

### 2.3 Converte o peso de Libras para KG

In [51]:
def convert_weight_kg(w):
    if pd.isna(w): return None
    if 'kg' in str(w): return float(str(w).replace('kg', ''))
    try:
        val = float(re.search(r"(\d+)", str(w)).group(1))
        return round(val * 0.453592, 1)
    except:
        return None

### 2.4 Remove as estrelas que estão presentes nas colunas W/F, SM e IR

In [52]:
def clean_stars(val):
    """Remove o caractere ★ e converte para inteiro"""
    if pd.isna(val): return 0
    return int(str(val).replace('★', '').strip())


## 3. Carregando os dados

Iniciando o processo ETL carregando o arquivo CSV com dados não processados.

In [53]:
df = pd.read_csv(INPUT_FILE)
print(f"   Carregado: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
print(f"   Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

   Carregado: 18,979 linhas x 77 colunas
   Memoria: 33.84 MB


/tmp/ipykernel_11117/1066572168.py:1: DtypeWarning: Columns (76) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(INPUT_FILE)


### 3.1 Remoção de Colunas indesejadas
Colunas como Url da imagem do jogador ou Status que já existe no csv não são necessárias, assim como data de empréstimo do jogador sendo que só existe uma data final ## VERIFICAR

In [54]:
url_cols = [col for col in df.columns if 'Url' in col or 'url' in col]
cols_to_drop = url_cols + ['Loan Date End']
df = df.drop(columns=cols_to_drop)

### 3.2 Uso das funções de Limpeza

In [55]:
# Limpeza de Estrelas (IR, W/F, SM)
star_cols = ['IR', 'W/F', 'SM']
for col in star_cols:
    df[col] = df[col].apply(clean_stars)
    
# Formatação de Data (Joined)
# Converte 'Jul 1, 2004'para o formato ISO
df['Joined'] = pd.to_datetime(df['Joined']).dt.strftime('%Y-%m-%d')

# Transformar Altura e Peso
df['Height'] = df['Height'].apply(convert_height_cm)
df['Weight'] = df['Weight'].apply(convert_weight_kg)
df = df.rename(columns={'Height': 'Height_cm', 'Weight': 'Weight_kg'})

# Converter Valores Financeiros
finance_cols = ['Value', 'Wage', 'Release Clause','Hits']
for col in finance_cols:
    df[col] = df[col].apply(convert_m_and_k)

### 3.3 Separação de time e contrato em 2 colunas diferentes

In [56]:
def split_team_contract(text):
    lines = [line.strip() for line in str(text).split('\n') if line.strip()]
    return lines[0], (lines[1] if len(lines) > 1 else "N/A")

temp_team_contract = df['Team & Contract'].apply(split_team_contract)
df['Team'] = [x[0] for x in temp_team_contract]
df['Contract_Period'] = [x[1] for x in temp_team_contract]
df = df.drop(columns=['Team & Contract'])

## 4. Salvando os Dados Processados

In [57]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
df.to_csv(os.path.join(OUTPUT_DIR, OUTPUT_FILE_CSV), index=False)

print(f"Processamento finalizado!")

Processamento finalizado!


In [58]:
DB_CONFIG = {
    'host': 'localhost',
    'port': 5433, 
    'database': 'postgres', 
    'user': 'airflow',
    'password': 'airflow'
}

def ingest_to_postgres(df_silver):
    print("Iniciando ingestão completa no PostgreSQL...")
    
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        
        data = []
        for _, row in df_silver.iterrows():
            data.append((
                int(row['ID']), 
                str(row['LongName']), 
                str(row['Name']), 
                str(row['Nationality']),
                str(row['Positions']), 
                int(row['Age']), 
                int(row['↓OVA']), 
                int(row['POT']),
                str(row['Team']), 
                str(row['Contract_Period']), 
                float(row['Height_cm']),
                float(row['Weight_kg']), 
                str(row['foot']), 
                int(row['BOV']), 
                str(row['BP']),
                int(row['Growth']), 
                row['Joined'], 
                float(row['Value']), 
                float(row['Wage']),
                float(row['Release Clause']), 
                int(row['Attacking']), 
                int(row['Crossing']),
                int(row['Finishing']), 
                int(row['Heading Accuracy']), 
                int(row['Short Passing']),
                int(row['Volleys']), 
                int(row['Skill']), 
                int(row['Dribbling']),
                int(row['Curve']), 
                int(row['FK Accuracy']), 
                int(row['Long Passing']),
                int(row['Ball Control']), 
                int(row['Movement']), 
                int(row['Acceleration']),
                int(row['Sprint Speed']), 
                int(row['Agility']), 
                int(row['Reactions']),
                int(row['Balance']), 
                int(row['Power']), 
                int(row['Shot Power']),
                int(row['Jumping']), 
                int(row['Stamina']), 
                int(row['Strength']),
                int(row['Long Shots']), 
                int(row['Mentality']), 
                int(row['Aggression']),
                int(row['Interceptions']), 
                int(row['Positioning']), 
                int(row['Vision']),
                int(row['Penalties']), 
                int(row['Composure']), 
                int(row['Defending']),
                int(row['Marking']), 
                int(row['Standing Tackle']), 
                int(row['Sliding Tackle']),
                int(row['Goalkeeping']), 
                int(row['GK Diving']), 
                int(row['GK Handling']),
                int(row['GK Kicking']), 
                int(row['GK Positioning']), 
                int(row['GK Reflexes']),
                int(row['Total Stats']), 
                int(row['Base Stats']), 
                int(row['W/F']),
                int(row['SM']), 
                str(row['A/W']), 
                str(row['D/W']), 
                int(row['IR']),
                int(row['PAC']), 
                int(row['SHO']), 
                int(row['PAS']), 
                int(row['DRI']),
                int(row['DEF']), 
                int(row['PHY']), 
                int(row['Hits'])
            ))

        insert_sql = """
            INSERT INTO silver.fifa21_players (
                player_id, long_name, name, nationality, positions, age, overall_rating, 
                potential_rating, team, contract_period, height_cm, weight_kg, 
                preferred_foot, best_overall_rating, best_position, growth, joined_date, 
                value_eur, wage_eur, release_clause_eur, attacking_total, crossing, 
                finishing, heading_accuracy, short_passing, volleys, skill_total, 
                dribbling, curve, fk_accuracy, long_passing, ball_control, 
                movement_total, acceleration, sprint_speed, agility, reactions, 
                balance, power_total, shot_power, jumping, stamina, strength, 
                long_shots, mentality_total, aggression, interceptions, positioning, 
                vision, penalties, composure, defending_total, marking, 
                standing_tackle, sliding_tackle, goalkeeping_total, gk_diving, 
                gk_handling, gk_kicking, gk_positioning, gk_reflexes, total_stats, 
                base_stats, weak_foot, skill_moves, attack_work_rate, defense_work_rate, 
                international_reputation, pace, shooting, passing, dribbling_stat, 
                defending_stat, physical, hits
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
                %s, %s, %s, %s, %s, %s, %s
            )
            ON CONFLICT (player_id) DO UPDATE SET
                overall_rating = EXCLUDED.overall_rating,
                value_eur = EXCLUDED.value_eur,
                team = EXCLUDED.team;
        """

        execute_batch(cur, insert_sql, data, page_size=1000)
        conn.commit()
        print(f"Ingestão finalizada com sucesso! {len(data)} registros na silver.fifa21_players.")

    except Exception as e:
        print(f"Erro na ingestão: {e}")
        if 'conn' in locals(): conn.rollback()
    finally:
        if 'cur' in locals(): cur.close()
        if 'conn' in locals(): conn.close()

ingest_to_postgres(df)

Iniciando ingestão completa no PostgreSQL...
Ingestão finalizada com sucesso! 18979 registros na silver.fifa21_players.
